## Some basic plotting experiments

This notebook contains some basic plotting experiments, and explores the capabilities and syntax of the matplotlib library.  Matplotlib is a very capable library for 2-dimensional visualization, although it can sometimes take a fair bit of effort to make the plots exactly as one might desire. 

## Plotting in two dimensions with matplotlib.

A useful resource for the basics of matplotlib is the [matplotlib FAQ "General Concepts" section](http://matplotlib.org/devdocs/faq/usage_faq.html#general-concepts).  It outlines the primary structures and terminology used by matplotlib.  We summarize this below. 

The **matplotlib.pyplot** module is the core library for producing 2-dimensional plots in matplotlib.  A display produced by matplotlib is called a **figure**, and figures have potentially many parts, called **axes**. 

<img src="fig_map.png" alt="Riemann sum example" width=300 height=300 alt="taken from http://matplotlib.org/devdocs/faq/usage_faq.html#general-concepts">

The **axes** belong to the figure.  matplotlib has a high-level graphics-display object called an **artist** and all objects (figures, axes, axis, text, etc) are artist objects. 

matplotlib expects all array-objects to be **numpy** arrays.  Other array types can work in matplotlib but often these create problems. 

matplotlib documentation distinguishes between the **backend** and the **frontend**. 
    
- The frontend refers to the way in which you generate code for matplotlib.  For us, this is the i-python notebook.  
- The backend refers to how one turns the code into graphics, or potentially an interactive environment. There are two primary backend types for matplotlib:
- Hardcopy backends.  These generate static image files from your code. 
- Interactive backends.  These generate code (some generate and execute the code) for interactive graphics.  For example, some backends generate java code that can be integrated with a web-page to render your application on-line.
        
For most tasks we will use the default backend for matplotlib.  This requires no special actions. We will also explore applications that use other backends. 

### Simple plot types with matplotlib

Let's start off making a basic figure.


In [ ]:
## let's begin with a basic plot that we can dress-up later.
import numpy as np
import matplotlib.pyplot as plt
%matplotlib nbagg

## the data
dt = 0.001 ## how closely the x-coordinates are spaced
t = np.arange(0.0, 2*np.pi, dt) ## x-coordinates of the plot
y = np.sin(t) + (0.2+0.18*np.cos(13*t))*np.sin(36*t) ## y-oordinates of the plot

## the plot
plt.plot(t,y) # the x-axis coordinates come from the t-array. The y-axis from y-array.
plt.xlabel('time')
plt.ylabel('some complicated function of time, close to y=sin(t)')
plt.title('A very basic plot')

Criticisms or standard things one might want to change in a plot such as the above:
    
    1) It's too small.  
    2) Notice the "box" is not cleanly aligned to the data on the right-side.
    3) Let's make the text biggger. 
    4) Let's add a box in the top-right that says something, say if you wanted to insert a legend, or insert additional details for some other reason. 
    5) Let's make the y-axis text fit the box. 

In [ ]:
fig = plt.figure() ## get the figure-object for our plot.
fig.set_size_inches(10,10) ## set the dimensions for the figure

plt.plot(t,y) ## use "fontsize" to adjust size of font.
plt.xlabel('time', fontsize=20)
plt.ylabel('some complicated function of time,\n close to y=sin(t)', fontsize=20)
plt.title('A very basic plot', fontsize=20)

## Here we set the **axis** (not axes!).  axis are just the
##  bounding box and decorations associated to our figure. 
xpad=0.1 ## gap between graph and box on left/right
ypad=0.2 ## gap between graph and box top/bot
plt.axis([0-xpad, 2*np.pi+xpad, np.amin(y)-ypad, np.amax(y)+xpad])
plt.plot(t,y)
plt.xticks([i*np.pi/4 for i in range(9)]) ## set the bottom ticks to be
  ## multiples of pi. 

ax1 = plt.axes([0.59,0.59, 0.3, 0.3], axisbg='y')
## the first two coordinates are the (x,y) coordinates
## of the bottom-left of our red box.  The second two
## are the width and height.  All are given int terms of
## the dimensions of the underlying figure, with dimensions
## measured from 0 to 1.  There are routines that allow you 
## to convert between these "natural figure dimensions" and
## the dimensions of your data.  
plt.xticks([1.0, 5.5]) ## change location of the tick marks
plt.yticks([0.3, 1.5])
plt.plot(t,y**2)
plt.xlabel("plot of y^2 vs the same t")

Sometimes your data will have details you will want to see, but a standard linear scale (eg $\{0, 1, 2, 3, 4, \cdots\}$ all spaced equally) does not suffice. 

In [ ]:
dt = 0.001 ## how closely the x-coordinates are spaced
t = np.arange(0.0, 2*np.pi, dt) ## x-coordinates of the plot
y = (np.sin(t))**32+ 0.01
plt.plot(t,y) ## standard linear plot
#plt.semilogy(t,y) ## log plot on only the y-axis
 ## there is also a semilogx plot call
#plt.loglog(t,y) ## logarithm of both the x and y-axis
plt.title("Plot with details happening at various scales.")

In [ ]:
## somewhat less related is the "plot with error bars" command
## for this, we pass the errors in the x and y coordinates as lists

dt = 0.4 ## how closely the x-coordinates are spaced
t = np.arange(0.0, 2*np.pi, dt) ## x-coordinates of the plot
y = (np.sin(t))**32+0.1
ax1 = plt.subplot() ## another routine to request axis, and specify location
#ax1.set_yscale("log")  ## you can set the scale using the axes.

yerrarr = np.zeros_like(t)
## let's make the y errors somewhat irregular
for i in range(len(yerrarr)): yerrarr[i] = 0.04+0.06*np.sin(8*np.pi*i/float(len(yerrarr)))**2

## the plot takes the t coordinates, y coordinates, x-error bars as an array, 
## and y-error bars as an array.
plt.errorbar(t,y, xerr=np.zeros_like(t)+0.1, yerr=yerrarr, ecolor='r')
plt.title("y(t) = sin^32(t)+1/10", fontsize=20)
plt.xlabel("t-axis")
plt.ylabel("y-axis")

You can also use LaTex in your text. 

In [ ]:
## Let's repeat a previous plot, but with some LaTeX. 
fig = plt.figure(3) 
fig.set_size_inches(10,10) 

dt = 0.001 ## how closely the x-coordinates are spaced
t = np.arange(0.0, 2*np.pi, dt) ## x-coordinates of the plot
y = np.sin(t) + (0.2+0.18*np.cos(13*t))*np.sin(36*t) ## y-oordinates of the plot

plt.plot(t,y) 

## this tells matplotlib to use LaTeX to construct all the text!
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

plt.xlabel('time', fontsize=20)
plt.ylabel('some complicated function of time,\n close to $y=\sin(t)$', fontsize=20)
plt.title("A very basic plot using \LaTeX\n"
          "handy if you need to describe formulas like \n"
          r"$y=\sin t+(\frac{1}{5}+\frac{9}{50}\cos(13t))\sin(36t)$", fontsize=20)
## in the above the r before the quote indicates a "raw" string, making 
## escape sequences, such as \n disabled.  This is because it would interpret
## \f as an escape sequence.  You could also type a regular string and write
## \\frac instead of \frac

xpad=0.1 ## gap between graph and box on left/right
ypad=0.2 ## gap between graph and box top/bot
plt.axis([0-xpad, 2*np.pi+xpad, np.amin(y)-ypad, np.amax(y)+xpad])
plt.plot(t,y)
plt.xticks([i*np.pi/4 for i in range(9)]) ## set the bottom ticks to be
  ## multiples of pi. 

ax1 = plt.axes([0.59,0.59, 0.3, 0.3], axisbg='y')
plt.xticks([1.0, 5.5]) ## change location of the tick marks
plt.yticks([0.3, 1.5])
plt.plot(t,y**2)
plt.xlabel("plot of $y^2$ vs the same $t$", fontsize=16)

## Other kinds of 2d plots in matplotlib. 

matplotlib lets you do any kind of 2d plot, the limit being only your imagination and the amount of time you are willing to put into the process.  But matplotlib also has a few helper-routines to make certain kinds of plots relatively easy-to-do.  We demonstrate below:

    - shapes - produces standard shapes like arrows, regular polygons, boxes with rounded corners, circles, filled closed splines, etc. 
    - histograms 
    - contour plots 
    - pie charts 
    - polar coordinate plots
    

In [ ]:
## a shape example: arrows, rounded rectanges, etc. 
import matplotlib.patches as mpat

fig, ax = plt.subplots()

## first pair are bottom-left coordinates before rectangle is "rounded"
##  the second numbers are the width and height, like a regular rectangle.
##
## the pad number is the radius of the rounding, think of this as a little
## disc of this radius being run with its centre along the original rectangle
## boundary. 
##
## fc is the filling color.  ec is the boundary color.
rbox = mpat.FancyBboxPatch( (0.2,0.2), 0.4, 0.4, boxstyle="round,pad=0.1",
                           fc=(1.0, 0.6, 0.4), 
                           ec=(0.2, 0.8, 1.0) )
ax.add_patch(rbox)

ax.text(0.25, 0.35, 'Rounded Box', size=16)


In [ ]:
## a spline example
import matplotlib.patches as mpat
import matplotlib.path as mpth

fig, ax = plt.subplots()

P = mpth.Path
path_data = [
    (P.MOVETO, (0,0)),
    (P.LINETO, (0,3)),
    (P.CURVE4, (2.5,3)),
    (P.CURVE4, (-2, 1.5)), 
    (P.LINETO, (1.4,0)),
    (P.CLOSEPOLY, (0,0)),
    ]
codes,verts = zip(*path_data)
path = mpth.Path(verts,codes)
patch = mpat.PathPatch(path, facecolor='r', alpha=0.5)
ax.add_patch(patch)

x,y=zip(*path.vertices)
line, = ax.plot(x,y,'go-')
ax.grid()
ax.axis('equal')

In [ ]:
## A histogram example. 
## 
## let's take a sampling of points in the plane and draw a histogram
##  of the projection of the data onto a given axis. 

## multi-variable normal. centre, covariance matrix, num pts.
data = np.random.multivariate_normal([0,0],[[1,0],[0,2]],  
                                     800)

fig,ax = plt.subplots()
## this helps me force a 1:1 aspect ratio in the figure.
xw = np.max(data.T[0]) - np.min(data.T[0])
yw = np.max(data.T[1]) - np.min(data.T[1])
mw = max(xw,yw)
xw = 8*xw/mw ## the largest dimension is 10. 
yw = 8*yw/mw
fig.set_size_inches(xw,yw)

## let's project onto the line with equation
##  ax+by = 0
a = 2
b = 1
## make the line go to roughly the bounding box.
bounds = []
if b!=0: bounds.append(abs(xw/b))
if a!=0: bounds.append(abs(yw/a))
M = min(bounds)
plt.plot(b*np.arange(-M,M, 0.1), -a*np.arange(-M,M, 0.1), 'b-')
plt.plot([b*M], [-a*M], 'yo')

## data plot
plt.plot(data.T[0], data.T[1], 'ro')
plt.title("Normally distributed data and the "+str(a)+"x+"+str(b)+"y=0 line")
## histogram
nPV = 1/np.sqrt(b**2+a**2)
projDat = [nPV*(data[i,0]*b-data[i,1]) for i in range(data.shape[0])]
ax1=plt.axes([0.95,0.35,0.4,0.3], axisbg='y')
plt.hist(projDat, normed=True, bins=30)
plt.title("histogram of data projected\n onto the "+str(a)+"x+"+str(b)+"y=0 line")


In [ ]:
## a contour plot.  Let's try a typical multi-variable calculus
##  plot of level-sets and gradient vector field
fig, ax = plt.subplots() 
fig.set_size_inches(10,10) 

import sympy as sp
x,y = sp.symbols('x y')
f = x*y
#f = x**3-x*y**2
#f = x**4-6*x**2*y**2+y**4
## try a few of the others, these are the real part of z^n = (x+iy)^n for n=2,3,4. 

grad = [sp.diff(f,x), sp.diff(f,y)]

import matplotlib.mlab as mlab
Z = sp.lambdify( (x,y), f, "numpy")
Zx = sp.lambdify( (x,y), grad[0], "numpy")
Zy = sp.lambdify( (x,y), grad[1], "numpy")

Y, X = np.mgrid[-2:2:100j, -2:2:100j] ## x and y coordinates of a 2-dimension array of points
 ## in the plane

speed = np.sqrt(Zx(X,Y)**2 + Zy(X,Y)**2)
mspd = speed.max()
    
CP = plt.contour(X,Y,Z(X,Y), 30) ## 30 contours plotted.
plt.clabel(CP, inline=1, fontsize=14) ## this add the level-set number to the curve

## the main tool for plotting vector fields is the quiver object
## it allows you to set a variety of properties of the vectors
##  pivot is where you centre your plotted vector, i.e. do you put the tail, head or
##  some other point at the X,Y coordinate for the vector field Zx(X,Y), Zy(X,Y)
Xs = X[::6,::6]
Ys = Y[::6,::6]
ax.quiver(Xs,Ys,Zx(Xs,Ys), Zy(Xs,Ys), scale_units='xy', scale=10)

## I find the flow lines easier to process than the actual 
##  vector field.  streamplot gives us the flow lines.
#ax.streamplot(X, Y, Zx(X,Y), Zy(X,Y), color='0.2', density=0.5, 
#              linewidth=5*(speed/mspd) )

plt.title("Level sets of $"+sp.latex(sp.Eq(sp.Symbol('f(x,y)'), f) )+"$\n"
          "and the gradient", fontsize=26)


A pie chart, using the [xkcd](http://xkcd.com) style.

In [ ]:
## unfortunately the xkcd style does not allow for latex, so we disable
## latex text output. 
plt.rc('text', usetex=False)

with plt.xkcd(): ## this enables the xkcd style.
    
    fig=plt.figure()
    fig.set_size_inches(10,10) 
    
    fracs = [15,30,45,10]
    pushOut = (0,0.1,0,0) ##
    labL = 'middling pie slice', 'removed pie slice', 'big pie slice', 'teeny pie slice'
    
    ## explode is the push-out parameter.  labels are the text around the
    ## circumference.  autopct puts the percentage in the pie slices. 
    ## shadow adds a bit of depth to the image.
    plt.pie(fracs, explode=pushOut, labels=labL, autopct='%1.1f%%', shadow=False)
    ## shadow=True / False
    plt.title('A generic pie chart, with the 2nd segment popped out.', fontsize=20)
    

In [ ]:
## polar coordinates

theta = np.arange(0, 30*2*np.pi, 0.01)
r = np.log(theta)

ax = plt.subplot(projection='polar')
fig = plt.gcf()
fig.set_size_inches(10,10)

plt.plot(theta, r)
plt.title("graph of $r=\\log(\\theta)$\n"
          "on the interval $\\theta \in [0,60\pi]$", fontsize=26)

## Interactive graphics. 

The easiest-to-use library for interactive graphics in the i-python notebook that I am aware of is the **IPython.html library**. 

This library provides convenient and easy-to-code interactive sessions. 

In [ ]:
## This example provides a range of 
from IPython.html.widgets import interact

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

I=np.linspace(0,2*np.pi,300) ## the interval [0,2π]

def pltsin(k):
    plt.plot(I, np.sin(k*I))
    plt.title("Plot of sin(kx) on interval [0,2π], k="+str(k))

## this routine calls the pltsin function whenever the user
##  updates the slider.  k=(1,10,0.1) allows the user to set
## the argument k in the pltsin function between 1 and 10, 
##  with a step-size of 0.1. 
interact(pltsin, k=(1,10,0.1))

### Let's step-up the complexity and make an interactive solution to the mid-term exam. 

Recall the single pendulum with friction

$$\theta'' + a \theta' + b \sin \theta = 0.$$

We approximate solutions by first converting the differential equation to a 1st-order ODE:

$$\pmatrix{\theta \cr \theta'}' = \pmatrix{\theta' \cr -a\theta'-b\sin \theta}$$

We then recursively solve for the vectors $$\pmatrix{\theta(t_{i+1}) \cr \theta'(t_{i+1})} = \pmatrix{ \theta(t_i) \cr \theta'(t_i) } + \Delta t \pmatrix{ \theta'(t_i) \cr -a \theta'(t_i) - b\sin \theta(t_i)}$$

which is called *Euler's Method* with step size $\Delta t = t_{i+1} - t_i$. 

In [ ]:
## let's start off by grabbing the code from the exam solutions. 
import math
## I = interval of approximation. 
## n = number of steps
## t0 = initial theta value at I[0]
## d0 = initial theta' value at I[0]
def eulerApprox(I, n, t0, d0, a):
    domArr = [I[0]]
    tArr = [t0]
    tdArr = [d0]
    dt = (I[1]-I[0])/n
    for i in range(1,n):
        tt = tArr[-1] + dt*tdArr[-1]
        tdt = tdArr[-1] +( -a*tdArr[-1]-np.sin(tArr[-1]) )*dt
        tArr.append(tt)
        tdArr.append(tdt)
        domArr.append(domArr[-1]+dt)
    return [tArr, tdArr, domArr]

def makeplot(I1, n, t0, d0, a):
    [tArr, tdArr,domArr] = eulerApprox([0, I1], n,t0,d0,a)
    ## a faster pendulum
    [tap, tdp, dap] = eulerApprox([0,I1], n, t0, d0+1,a)
    ## a slower pendulum
    [tam, tdm, dam] = eulerApprox([0,I1], n, t0, d0-1,a)
    ## plot the pendulum curves
    plt.plot(domArr, tArr, 'k-')
    plt.plot(domArr, tap, 'r--')
    plt.plot(domArr, tam, 'g--')
    ## let's plot some relevant multiples of 2pi on the theta axis.
    maxt = np.max( [ np.max(tArr), np.max(tap), np.max(tam) ] )
    mint = np.min( [ np.min(tArr), np.min(tap), np.min(tam) ] )
    Mi = math.floor(maxt / (2*np.pi))
    Ni = math.floor(mint / (2*np.pi))
    if (mint - 2*np.pi*Ni > np.pi): Ni = Ni + 1
    if (maxt - 2*np.pi*Mi > np.pi): Mi = Mi + 1
    plt.plot(np.linspace(I1,I1,Mi-Ni+1), 
             np.linspace(Ni*2*np.pi, Mi*2*np.pi, Mi-Ni+1), 'bo')
    plt.title("Euler approximation to ODE\n"
              "on interval [0,"+str(I1)+"] with "+str(n)+" steps\n"
              "theta0 = "+str(t0)+"   thetap0 = "+str(d0))

interact(makeplot, I1=(4,20,0.5), n=(2, 200, 1), 
         t0=(-2*np.pi, 2*np.pi, 0.1), d0=(-20, 20, 0.1),
         a=(0.1, 2, 0.02))


For more sophisticated interactive graphics sessions, matplotlib has a widget library available.  

 Overview of interactive matplotlib sessions:
 
  * matplotlib interactive sessions are controled by objects it called "widgets". 
  * Widgets are routines that respond to user input, and call other routines that you define, when the user does specified things.  Some of the most basic widgets are:
      - **Button** represents an individual, pressable button.
      - **CheckButtons** a radiobutton, this allows you have several checkable buttons.
      - **Cursor** a widget that tracks and draws lines in your figure according to where you mouse pointer is located. 
      - **RadioButtons** where only one of several buttons can be pressed. 
      - **Slider** allows user to specify a float by sliding a bar. 

See the [matplotlib widget example page](http://matplotlib.org/1.5.1/examples/widgets/index.html) for detailed examples. 

In [ ]:
%matplotlib qt
## this sets the back-end. qt, wx, gtk, tk
##  for me qt seems to be what I need. 

import numpy as np
import pylab as pl
from matplotlib.widgets import Slider, Button, RadioButtons

ax = pl.subplot(111)
pl.subplots_adjust(left=0.25, bottom=0.25)
t = np.arange(0.0, 1.0, 0.001)
a0 = 5
f0 = 3
s = a0*np.sin(2*np.pi*f0*t)
l, = pl.plot(t,s, lw=2, color='red')
pl.axis([0, 1, -10, 10])

axcolor = 'lightgoldenrodyellow'
axfreq = pl.axes([0.25, 0.1, 0.65, 0.03], axisbg=axcolor)
axamp  = pl.axes([0.25, 0.15, 0.65, 0.03], axisbg=axcolor)

sfreq = Slider(axfreq, 'Freq', 0.1, 30.0, valinit=f0)
samp = Slider(axamp, 'Amp', 0.1, 10.0, valinit=a0)

def update(val):
    amp = samp.val
    freq = sfreq.val
    l.set_ydata(amp*np.sin(2*np.pi*freq*t))
    draw()
sfreq.on_changed(update)
samp.on_changed(update)

resetax = pl.axes([0.8, 0.025, 0.1, 0.04])
button = Button(resetax, 'Reset', color=axcolor, hovercolor='0.975')
def reset(event):
    sfreq.reset()
    samp.reset()
button.on_clicked(reset)

rax = pl.axes([0.025, 0.5, 0.15, 0.15], axisbg=axcolor)
radio = RadioButtons(rax, ('red', 'blue', 'green'), active=0)
def colorfunc(label):
    l.set_color(label)
    draw()
radio.on_clicked(colorfunc)

pl.show()

## Plotting in three dimensions with matplotlib.

matplotlib has some elements of 3-dimensional plotting but it has largely not been designed for this task.  Below we demonstrate one example, a parametric surface. 

Our surface will be equidistant from the circle

$$ R(\cos \theta, \sin \theta, 0) $$

in the $xy$-plane.  The variable $R$ we call $bigR$ in the code. The distance we take from this circle, we denote $r$ here, will be $smR$ in the code.  This gives us the parametrization of the surface:

$$ (R\cos \theta, R\sin \theta, 0) + r\cos \phi \left( \cos \theta, \sin\theta,0\right) + r \sin \phi \left(0,0,1\right)$$
which is
$$(\theta,\phi) \longmapsto \left( (R+r\cos\phi)\cos \theta, (R+r\cos \phi)\sin \theta, r \sin \phi \right)$$

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(12,12))

ax = fig.add_subplot(111, projection='3d')

t1 = np.linspace(0, 2*np.pi, 100)
t2 = np.linspace(0, 2*np.pi, 100)

## we use outer to form a doubly-indexed list of numbers.
## alternatively one could form the doubly-index list
## beforehand...
bigR = 1.0
smR = 0.2
x1 = np.outer( (bigR+smR*np.cos(t1)), np.cos(t2) )
y1 = np.outer( (bigR+smR*np.cos(t1)), np.sin(t2) )
z1 = np.outer( smR*np.sin(t1), np.ones(np.size(t2)) ) 

ax.set_title('A view of the tire tube.', fontsize=14)
ax.plot_surface(x1, y1, z1, rstride=5,cstride=5, color='r', shade=True)

## here is a fairly nice way to set a common aspect ratio
scl = np.array([getattr(ax,'get_{}lim'.format(dim))() for dim in 'xyz'])
ax.auto_scale_xyz(*[[np.min(scl),np.max(scl)]]*3)

## elev sets how far up from the xy-plane we arein.
## azim is the angle from the xz-plane
ax.view_init(elev=30,azim=45)
plt.show()